In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

driver = webdriver.Chrome(options=options)



In [ ]:
# 크롤링할 URL
url = "https://app.catchtable.co.kr/ct/curation/culinaryclasswars2?curationQuickFilterKey=53507&hasShopRefsFromClient=1&isUseExhibitionFilter=1&metaContractedType=1&currentExhibitionKey=classwars2-all&serviceType=INTEGRATION&sortMethod=price_level_desc&zoomLevel=10&centerBoundsLat=37.5404502815133&centerBoundsLng=126.97067134216111&isNewSearchInMap=1&isSearchedInMap=0&isShowMapSearchButton=0&subFilterGroupItem=%255B%255D&showTargetShopTooltip=0&isSettingTargetShopBound=0&prevSearchType=FILTER_SEARCH&location=CAT011_CAT011001_CAT011002_CAT011003_CAT011004_CAT011005_CAT011006_CAT011007_CAT011008_CAT011009_CAT011010&uniqueListId=1768350426690&shopClickCount=9&viewMode=LIST&renderMapComponent=1&showShopSlide=0"
driver.get(url)
time.sleep(3) # 페이지 초기 로딩 대기

In [117]:
# Selector 설정
CSS_CARD_CONTAINER = "div.l0xety0.l0xety2.l0xety4"
CSS_NAME = ".l0xety9"
CSS_CHIEF = ".vrh4k92 span"
CSS_RATING = ".oron0y1"
CSS_REVIEW_COUNT = ".oron0y2.oron0y4"
CSS_LOCATION_CATEGORY = "span._1nmepil1._1nmepil3"
CSS_STATUS_TIME = ".ruh8m12.ruh8m14.ruh8m16"
CSS_PRICE = ".ruh8m12.ruh8m13.ruh8m16"
CSS_MESHLIN = ".vrh4k93 span"

print("Selector 설정 완료")

Selector 설정 완료


In [118]:
# [로직 수정] '화면이 처음으로 돌아가는 현상' 방지
# 원인: 고정된 XPath(상단 영역)로 마우스를 계속 이동시키면 브라우저가 위로 스크롤합니다.
# 해결: 현재 보이는 '마지막 카드'로 마우스를 이동시킨 뒤 PageDown을 하여 계속 아래로 내려갑니다.

collected_data = {} 
no_new_data_count = 0

try:
    print("수집 시작...")
    
    while True:
        # 1. 데이터 수집
        visible_cards = driver.find_elements(By.CSS_SELECTOR, CSS_CARD_CONTAINER)
        new_data_found = False
        
        for card in visible_cards:
            try:
                try: name = card.find_element(By.CSS_SELECTOR, CSS_NAME).text
                except: continue
                
                if name in collected_data:
                    continue

                new_data_found = True
                
                # 정보 추출
                try: chief = card.find_element(By.CSS_SELECTOR, CSS_CHIEF).text
                except: chief = ""
                try: rating = card.find_element(By.CSS_SELECTOR, CSS_RATING).text
                except: rating = ""
                try: review_count = card.find_element(By.CSS_SELECTOR, CSS_REVIEW_COUNT).text
                except: review_count = ""
                try:
                    loc_cats = card.find_elements(By.CSS_SELECTOR, CSS_LOCATION_CATEGORY)
                    location_category = " · ".join([el.text for el in loc_cats if el.text.strip() and el.text != "·"])
                except: location_category = ""
                try: status_time = card.find_element(By.CSS_SELECTOR, CSS_STATUS_TIME).text
                except: status_time = ""
                try: price = card.find_element(By.CSS_SELECTOR, CSS_PRICE).text
                except: price = ""
                try:
                    meshlin_elements = card.find_elements(By.CSS_SELECTOR, CSS_MESHLIN)
                    meshlin = ", ".join([el.text for el in meshlin_elements if el.text.strip()])
                except: meshlin = ""

                collected_data[name] = {
                    "name": name,
                    "chief_info": chief,
                    "rating": rating,
                    "review_count": review_count,
                    "location_category": location_category,
                    "status_time": status_time,
                    "price": price,
                    "badge_info": meshlin
                }
            except: continue

        # 2. 종료 조건 체크
        if not new_data_found:
            no_new_data_count += 1
            if no_new_data_count >= 10: 
                print("수집 종료 (더 이상 새로운 데이터 없음)")
                break
        else:
            no_new_data_count = 0
            print(f"현재 수집된 데이터 개수: {len(collected_data)}개")
            
        # 3. 스크롤 (가장 마지막 요소로 이동 -> PageDown x 2)
        if visible_cards:
            last_card = visible_cards[-1] # 현재 보이는 마지막 카드로 이동
            try:
                actions = ActionChains(driver)
                actions.move_to_element(last_card).perform()
                time.sleep(0.5)
                
                # PageDown 입력
                actions.send_keys(Keys.PAGE_DOWN).pause(0.5).send_keys(Keys.PAGE_DOWN).perform()
                
                time.sleep(1.5)
            except Exception as e:
                print(f"스크롤 중 에러: {e}")
                # 실패 시 body를 대상으로 시도
                body = driver.find_element(By.TAG_NAME, "body")
                body.send_keys(Keys.PAGE_DOWN)
                time.sleep(.5)

except Exception as e:
    print(f"에러 관련 메시지: {e}")
    
print(f"총 {len(collected_data)}개 데이터 수집 완료")

수집 시작...
현재 수집된 데이터 개수: 3개
현재 수집된 데이터 개수: 6개
현재 수집된 데이터 개수: 9개
현재 수집된 데이터 개수: 12개
현재 수집된 데이터 개수: 15개
현재 수집된 데이터 개수: 19개
현재 수집된 데이터 개수: 23개
현재 수집된 데이터 개수: 27개
현재 수집된 데이터 개수: 31개
현재 수집된 데이터 개수: 35개
현재 수집된 데이터 개수: 39개
현재 수집된 데이터 개수: 43개
현재 수집된 데이터 개수: 46개
현재 수집된 데이터 개수: 49개
현재 수집된 데이터 개수: 52개
현재 수집된 데이터 개수: 56개
현재 수집된 데이터 개수: 60개
현재 수집된 데이터 개수: 64개
현재 수집된 데이터 개수: 68개
현재 수집된 데이터 개수: 72개
현재 수집된 데이터 개수: 76개
현재 수집된 데이터 개수: 80개
현재 수집된 데이터 개수: 84개
현재 수집된 데이터 개수: 88개
현재 수집된 데이터 개수: 92개
현재 수집된 데이터 개수: 96개
현재 수집된 데이터 개수: 100개
현재 수집된 데이터 개수: 104개
현재 수집된 데이터 개수: 108개
현재 수집된 데이터 개수: 112개
현재 수집된 데이터 개수: 116개
현재 수집된 데이터 개수: 118개
수집 종료 (더 이상 새로운 데이터 없음)
총 118개 데이터 수집 완료


In [119]:
# 결과 저장
df = pd.DataFrame(list(collected_data.values()))
display(df.head())
df.to_csv("catchtable_crawling_final.csv", index=False, encoding="utf-8-sig")

,name,chief_info,rating,review_count,location_category,status_time,price,badge_info
0,앰배서더 서울 풀만 호빈,흑백요리사2: 후덕죽,4.7,(96),장충동 · 중식,영업중 • 11:30-22:00,"점심, 저녁 동일가 11-40만원","흑백요리사2: 후덕죽, 미쉐린 1스타, 흑백요리사2"
1,이타닉 가든,흑백요리사2: 손종원,4.9,"(2,652)",역삼 · 코스요리,영업중 • 12:00-22:00,점심 25만원 • 저녁 37만원,"흑백요리사2: 손종원, Taste of Seoul, 미쉐린 1스타, 흑백요리사2"
2,SOIGNE,흑백요리사2: 이준,4.8,"(1,632)",신사 · 컨템포러리,영업중 • 12:00-22:00,점심 24만원 • 저녁 38만원,"흑백요리사2: 이준, Taste of Seoul, 미쉐린 2스타, 흑백요리사2"
3,레스토랑 온 ON,흑백요리사2: 영블러드,4.9,(388),청담 · 프랑스음식,영업중 • 12:00-22:00,점심 저녁 동일가 10 - 25만원,"흑백요리사2: 영블러드, 미쉐린 가이드 2025"
4,라망시크레,흑백요리사2: 손종원,4.8,"(2,709)",회현 · 컨템포러리,영업중 • 12:00-22:00,점심 17만원 • 저녁 27만원,"흑백요리사2: 손종원, 미쉐린 1스타, 흑백요리사2"


----
# 주소갖고오기

In [ ]:
from curl_cffi import requests
import pandas as pd
import time
import json

# ==========================================
# 1. 설정
# ==========================================
TARGET_CURATION_KEY = "classwars2-all"
url = "https://ct-api.catchtable.co.kr/api/v6/search/curation/list"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Content-Type": "application/json",
    "Referer": "https://app.catchtable.co.kr/",
    "Origin": "https://app.catchtable.co.kr",
    # 쿠키 (403 에러 발생 시 개발자 도구에서 새 쿠키로 교체 필요)
    "Cookie": "_gcl_au=1.1.56958207.1768291961; _hackle_hid=e2ecdc37-4231-478c-b3d6-8bbd45b5bfcf; _hackle_did_7dQgTKfweH0n436c9aJLVh84yOncuWxD=e2ecdc37-4231-478c-b3d6-8bbd45b5bfcf; _gid=GA1.3.1186047315.1768291965; airbridge_migration_metadata__catchtable=%7B%22version%22%3A%221.11.0%22%7D; ab180ClientId=9b12e792-7ce3-4a13-adeb-876798c35d60; airbridge_referrer_campaign_params__catchtable=google.adwords%24%24%7B%22channel%22%3A%22google.adwords%22%2C%22campaign%22%3A%2223339493589%22%2C%22campaign_id%22%3A%2223339493589%22%2C%22ad_group%22%3A%22%22%2C%22ad_group_id%22%3A%22%22%2C%22ad_creative%22%3A%22%22%2C%22ad_creative_id%22%3A%22%22%2C%22ad_creative_id%22%3A%22%22%2C%22term%22%3A%22%22%2C%22sub_id%22%3A%22x%22%2C%22sub_id_1%22%3A%22%22%2C%22sub_id_2%22%3A%22%22%2C%22sub_id_3%22%3A%22%22%7D; airbridge_referrer_campaign_params_cta_parameter__catchtable=%7B%7D; airbridge_referrer_campaign_params_url__catchtable=https%3A//app.catchtable.co.kr/ct/exhibition/2512_limited_the_final%3Fairbridge_referrer%3Dairbridge%253Dtrue%2526channel%253Dgoogle.adwords%2526campaign%253D23339493589%2526campaign_id%253D23339493589%2526ad_group%253D%2526ad_group_id%253D%2526ad_creative%253D%2526ad_creative_id%253D%2526term%253D%2526sub_id%253Dx%2526sub_id_1%253D%2526sub_id_2%253D%2526sub_id_3%253D%2526click_id%253DCjwKCAiA95fLBhBPEiwATXUsxL2VSCprFaWAlwr-WyxMqd19ekacyd5d9rfHKgc2RXC_OnwXrPNxABoClgcQAvD_BwE%2526gclid%253DCjwKCAiA95fLBhBPEiwATXUsxL2VSCprFaWAlwr-WyxMqd19ekacyd5d9rfHKgc2RXC_OnwXrPNxABoClgcQAvD_BwE%2526ad_type%253Dclick%26gad_source%3D1%26gad_campaignid%3D23339498608%26gbraid%3D0AAAAACuLLC0l2L-_q2iMIKvxERkwhs35c%26gclid%3DCjwKCAiA95fLBhBPEiwATXUsxL2VSCprFaWAlwr-WyxMqd19ekacyd5d9rfHKgc2RXC_OnwXrPNxABoClgcQAvD_BwE; airbridge_referrer_campaign_params_timestamp__catchtable=1768304915629; _gcl_gs=2.1.k1$i1768357823$u51298297; _gac_UA-117680739-4=1.1768357824.CjwKCAiA95fLBhBPEiwATXUsxDMDnk5Zk79hJgE0kcOXco5-WKtEUJ45HzPY7UpVpzlra5qxvrHbURoC3B4QAvD_BwE; _gcl_aw=GCL.1768357827.CjwKCAiA95fLBhBPEiwATXUsxDMDnk5Zk79hJgE0kcOXco5-WKtEUJ45HzPY7UpVpzlra5qxvrHbURoC3B4QAvD_BwE; _hackle_mkt_7dQgTKfw=%7B%7D; _ga=GA1.1.956063542.1768291961; airbridge_session__catchtable=%7B%22id%22%3A%22cb8bfe40-add9-4e29-89fc-e149f34c9f40%22%2C%22timeout%22%3A1800000%2C%22start%22%3A1768357494043%2C%22end%22%3A1768359431412%7D; _ga_95C07ZWW1T=GS2.1.s1768357482$o6$g1$t1768359512$j43$l0$h0; _hackle_session_id_eH0n436c9aJLVh84yOncuWxD=1768362613467.5e3e8762; _ga_9ENCGJ7C7P=GS2.1.s1768362613$o7$g0$t1768362613$j60$l0$h0; __cf_bm=_ZMbbh3qpTJy.O4IIqGiHsRrpO3rroqess8Hxl7URlo-1768362615-1.0.1.1-g.cJrnnyfW_VQCOw8ZIpkCnZKysMJs6NancQuLhtUMPTBNTNgZrg6pHS4hHWgE35cKNc3eEg3HxNRygRr8V0kP2xWw02411ZvzQ5xC3pmpU; _hackle_last_event_ts_eH0n436c9aJLVh84yOncuWxD=1768362615917"
}

# ==========================================
# 2. 수집 실행 (강제 패턴 적용)
# ==========================================
all_data = []
current_count = 0  # 0, 20, 40 ... 이렇게 증가할 숫자

print(f"수집 시작...")

while True:
    # -----------------------------------------------------------
    # ★ 핵심: 사용자님이 찾은 패턴대로 Offset 생성
    # -----------------------------------------------------------
    if current_count == 0:
        offset_str = "0" # 첫 페이지는 0.0
    else:
        # 20:99:91-20:37:27-20:0:0 형식으로 조립
        offset_str = f"{current_count}:99:91-{current_count}:37:27-{current_count}:0:0"
        
    print(f"\n[데이터 {current_count}번부터 요청 중...] Offset: {offset_str}")
    
    payload = {
        "paging": {"offset": offset_str, "size": 20},
        "divideType": "NON_DIVIDE",
        "curation": {"curationKey": TARGET_CURATION_KEY},
        "sort": {"sortType": "recommended"},
        "userInfo": {"clientGeoPoint": {"lat": 37.563398, "lon": 126.9863309}}
    }

    try:
        response = requests.post(url, headers=headers, json=payload, impersonate="chrome110")
        
        if response.status_code != 200:
            print(f"!!! 에러: {response.status_code} (Offset 문제일 수 있음)")
            break
            
        json_resp = response.json()
        inner_data = json_resp.get('data', {})
        shop_results = inner_data.get('shopResults', {})
        items = shop_results.get('shops', [])
        
        if not items:
            print(">>> 데이터가 비어있습니다. 수집을 종료합니다.")
            break
            
        print(f">>> {len(items)}개 확보 완료! (누적 {len(all_data) + len(items)}개)")

        # 데이터 파싱
        for item in items:
            meta = item.get('shopMeta', {})
            coords = meta.get('shopCoord', {})
            prices = meta.get('prices', {})
            
            shop_info = {
                '식당명': meta.get('shopName'),
                '카테고리': meta.get('foodKind'),
                '평점': meta.get('avgScore'),
                '리뷰수': meta.get('reviewCount'),
                '지역': meta.get('landName'),
                '전화번호': meta.get('shopPhone'),
                '저녁가격': prices.get('dinnerPriceText'),
                '위도': coords.get('lat'),
                '경도': coords.get('lon'),
                '고유ID': meta.get('shopRef')
            }
            all_data.append(shop_info)
            
        # 다음 페이지를 위해 카운트 20 증가
        current_count += 20
        time.sleep(1.5) # 안전하게 1.5초 대기

    except Exception as e:
        print(f"에러 발생: {e}")
        break

# 저장
if all_data:
    df = pd.DataFrame(all_data)
    df.to_csv("catchtable_full_data.csv", index=False, encoding="utf-8-sig")
    print(f"\n[완료] 총 {len(df)}개의 식당 정보를 저장했습니다!")
    print(df.tail()) # 마지막에 수집된 데이터 확인
else:
    print("\n[실패] 데이터가 없습니다.")

수집 시작...

[데이터 0번부터 요청 중...] Offset: 0
>>> 20개 확보 완료! (누적 20개)

[데이터 20번부터 요청 중...] Offset: 20:99:91-20:37:27-20:0:0
>>> 20개 확보 완료! (누적 40개)

[데이터 40번부터 요청 중...] Offset: 40:99:91-40:37:27-40:0:0
>>> 20개 확보 완료! (누적 60개)

[데이터 60번부터 요청 중...] Offset: 60:99:91-60:37:27-60:0:0
>>> 20개 확보 완료! (누적 80개)

[데이터 80번부터 요청 중...] Offset: 80:99:91-80:37:27-80:0:0
>>> 20개 확보 완료! (누적 100개)

[데이터 100번부터 요청 중...] Offset: 100:99:91-100:37:27-100:0:0
>>> 20개 확보 완료! (누적 120개)

[데이터 120번부터 요청 중...] Offset: 120:99:91-120:37:27-120:0:0
>>> 5개 확보 완료! (누적 125개)

[데이터 140번부터 요청 중...] Offset: 140:99:91-140:37:27-140:0:0
>>> 데이터가 비어있습니다. 수집을 종료합니다.

[완료] 총 125개의 식당 정보를 저장했습니다!
                   식당명                                          상세페이지_URL
0          유용욱 바베큐 연구소  https://app.catchtable.co.kr/ct/shop/yyw?type=...
1    IMOK Smoke Dining  https://app.catchtable.co.kr/ct/shop/imok?type...
2                라망시크레  https://app.catchtable.co.kr/ct/shop/lamantsec...
3  Original Numbers 청담  https://app.catchtable.c

In [8]:
df1 = pd.read_csv('catchtable_crawling_final.csv')
df2 = pd.read_csv('catchtable_full_data_with_url.csv')


In [10]:
df1

,name,chief_info,rating,review_count,location_category,status_time,price,badge_info
0,앰배서더 서울 풀만 호빈,흑백요리사2: 후덕죽,4.7,(96),장충동 · 중식,영업중 • 11:30-22:00,"점심, 저녁 동일가 11-40만원","흑백요리사2: 후덕죽, 미쉐린 1스타, 흑백요리사2"
1,이타닉 가든,흑백요리사2: 손종원,4.9,"(2,652)",역삼 · 코스요리,영업중 • 12:00-22:00,점심 25만원 • 저녁 37만원,"흑백요리사2: 손종원, Taste of Seoul, 미쉐린 1스타, 흑백요리사2"
2,SOIGNE,흑백요리사2: 이준,4.8,"(1,632)",신사 · 컨템포러리,영업중 • 12:00-22:00,점심 24만원 • 저녁 38만원,"흑백요리사2: 이준, Taste of Seoul, 미쉐린 2스타, 흑백요리사2"
3,레스토랑 온 ON,흑백요리사2: 영블러드,4.9,(388),청담 · 프랑스음식,영업중 • 12:00-22:00,점심 저녁 동일가 10 - 25만원,"흑백요리사2: 영블러드, 미쉐린 가이드 2025"
4,라망시크레,흑백요리사2: 손종원,4.8,"(2,709)",회현 · 컨템포러리,영업중 • 12:00-22:00,점심 17만원 • 저녁 27만원,"흑백요리사2: 손종원, 미쉐린 1스타, 흑백요리사2"
...,...,...,...,...,...,...,...,...
113,윤주당,흑백요리사2: 술 빚는 윤주모,4.7,(68),해방촌 · 요리주점,오늘(수) 휴무,점심 영업안함 • 저녁 1 - 4만원,"흑백요리사2: 술 빚는 윤주모, 흑백요리사2"
114,보타르가 비노,흑백요리사2: 메이드 인 코리아,4.9,(171),청담 · 이탈리아음식,영업전 • 17:00 영업 시작,점심 영업안함 • 저녁 가격변동,"흑백요리사2: 메이드 인 코리아, 신규입점"
115,크라운돼지,흑백요리사2: 송훈,4.5,(24),"신사 · 육류,고기요리",영업전 • 17:00 영업 시작,점심 영업안함 • 저녁 1 - 4만원,"흑백요리사2: 송훈, 예약 성공 보장, 신규입점, 흑백요리사2"
116,신안가옥,흑백요리사2: 황금손,4.6,(166),"논현 · 육류,고기요리",영업중 • 12:00-23:00,점심 가격변동 • 저녁 가격변동,"흑백요리사2: 황금손, 신규입점"


In [11]:
# 1. 보기 편하게 변수로 저장
names_df1 = set(df1['name'])      # 원본 리스트 (예: 흑백요리사 정보)
names_df2 = set(df2['식당명'])     # 크롤링한 리스트

# 2. df1(원본)에는 있는데, df2(크롤링)에는 없는 것 찾기 (누락된 식당)
not_crawled = names_df1 - names_df2

# 3. df2(크롤링)에는 있는데, df1(원본)에는 없는 것 찾기 (이름이 미묘하게 다른 경우 등)
unexpected_crawled = names_df2 - names_df1

print(f"=== [주의] df1엔 있지만 df2엔 없는 것 ({len(not_crawled)}개) ===")
# 보기 좋게 리스트로 변환해서 출력
print(list(not_crawled))

print(f"\n=== [참고] df2엔 있지만 df1엔 없는 것 ({len(unexpected_crawled)}개) ===")
print(list(unexpected_crawled))

=== [주의] df1엔 있지만 df2엔 없는 것 (10개) ===
['카덴', '떡산 안국', '몽도 (Mòndo)', '소울 SOUL', '우동 카덴', '고미태', '천상현의 천상', '떡산 롯데백화점잠실점', '아선재', '떡산']

=== [참고] df2엔 있지만 df1엔 없는 것 (17개) ===
['몽도 (Mòndo) ', '옥동식 그릴 송파하남', '외포리 숯불장어', '오이너구리', '죠죠 트리플스트리트점', '급이다른부대찌개', '오코메', '힙한식', '무탄 파주 스타필드점', '동경밥상 제주점', '죠죠 판교아브뉴프랑점', '스시사카바 갓포아키', '더이탈리안 클럽 판교 테크원점', '최강달인 김혜규의 뼈대있는 짬뽕', '크라운돼지 제주점', '동경밥상 본점', '소울 SOUL ']


df2 = 주소정보 들어있음
df1 = 가게 정보들어있음

df1엔 있지만 df2엔 없는 것 => 여기있는 것들은 주소가 없다는 거겠네
df2엔 있지만 df1엔 없는 것 => 여기는 서울이 아닌 데이터가 있겠네

In [130]:
df2.head()

,식당명,카테고리,평점,리뷰수,지역,전화번호,저녁가격,위도,경도,고유ID
0,유용욱 바베큐 연구소,바베큐,4.8,268,남영,01039934729,저녁 15만원,37.543957,126.972782,WgsrWRcE8EOpoYVDzuBveA
1,IMOK Smoke Dining,바베큐,4.8,1488,신사,01039934729,저녁 15-22만원,37.521126,127.019125,GJ-Ffyc0hHpuuZ9ho_JS6A
2,라망시크레,컨템포러리,4.8,2709,회현,050712835380,저녁 27만원,37.559666,126.979542,VsD5rduELmS-Gh8gamV8Zw
3,Original Numbers 청담,코스요리,4.8,466,청담,050720889255,저녁 13만원,37.525096,127.040738,OyFrA7s3f89jANSiyNMtwg
4,앰배서더 서울 풀만 호빈,중식,4.7,96,장충동,050712835474,"점심, 저녁 동일가 11-40만원",37.560592,127.002094,LHkJtWrAKiexV14Xg9BWYQ


In [107]:
mergedata = pd.merge(df2,df1, how='outer', left_on = '식당명',right_on= 'name')

In [12]:
추가위치 = pd.read_csv('추가위치데이터.csv')
추가위치.head()

,식당명,카테고리,평점,리뷰수,지역,전화번호,저녁가격,위도,경도
0,고미태,일식당/라멘,4.5,326,서울 마포구 합정동 (월드컵로 41 1층),010-4876-8432,10000,37.5530,126.9119
1,카덴,이자카야/일식,4.4,정보없음,서울 서대문구 연희동 (연희로 173 1층),02-3142-6362,28000,37.5696,126.9278
2,떡산,떡볶이/분식,4.5,정보없음,서울 은평구 불광동 (연서로 247-1),0507-1479-4321,3500,37.6190,126.9284
3,떡산 롯데백화점잠실점,떡볶이/분식,정보없음,정보없음,서울 송파구 잠실동 롯데백화점 잠실점 B1,02-900-8592,3500,37.5131,127.1025
4,천상현의 천상,중식,4.5,정보없음,서울 서초구 양재동 (매헌로 16),02-정보없음,15000,37.4834,127.0337


In [13]:
추가위치.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   식당명     10 non-null     object 
 1   카테고리    10 non-null     object 
 2   평점      10 non-null     object 
 3   리뷰수     10 non-null     object 
 4   지역      10 non-null     object 
 5   전화번호    10 non-null     object 
 6   저녁가격    10 non-null     int64  
 7   위도      10 non-null     float64
 8   경도      10 non-null     float64
dtypes: float64(2), int64(1), object(6)
memory usage: 852.0+ bytes


In [14]:
concat_data = pd.concat([df2.drop('고유ID',axis=1),추가위치],ignore_index= True)

In [17]:
concat_data

,식당명,카테고리,평점,리뷰수,지역,전화번호,저녁가격,위도,경도,URL_Alias,서비스유형,상세페이지_URL
0,유용욱 바베큐 연구소,바베큐,4.8,268,남영,01039934729,저녁 15만원,37.543957,126.972782,yyw,DINING,https://app.catchtable.co.kr/ct/shop/yyw?type=...
1,IMOK Smoke Dining,바베큐,4.8,1488,신사,01039934729,저녁 15-22만원,37.521126,127.019125,imok,DINING,https://app.catchtable.co.kr/ct/shop/imok?type...
2,라망시크레,컨템포러리,4.8,2709,회현,050712835380,저녁 27만원,37.559666,126.979542,lamantsecret,DINING,https://app.catchtable.co.kr/ct/shop/lamantsec...
3,Original Numbers 청담,코스요리,4.8,468,청담,050720889255,저녁 13만원,37.525096,127.040738,original_numbers,DINING,https://app.catchtable.co.kr/ct/shop/original_...
4,앰배서더 서울 풀만 호빈,중식,4.7,96,장충동,050712835474,"점심, 저녁 동일가 11-40만원",37.560592,127.002094,theambassador_haobin,DINING,https://app.catchtable.co.kr/ct/shop/theambass...
...,...,...,...,...,...,...,...,...,...,...,...,...
130,떡산 안국,떡볶이/분식,정보없음,정보없음,서울 종로구 재동 (북촌로 14),0507-1309-1014,6000,37.579400,126.985100,NaN,NaN,NaN
131,아선재,한정식,4.3,정보없음,서울 강남구 대치동 (영동대로 333 지하1층),0507-1323-1114,120000,37.506200,127.063000,NaN,NaN,NaN
132,우동 카덴,우동/일식,4.4,정보없음,서울 서대문구 연희동 (연희로 173 1층),02-337-6360,9000,37.569600,126.927800,NaN,NaN,NaN
133,소울 SOUL,컨템퍼러리/한식,미슐랭1스타,정보없음,서울 용산구 해방촌 (신흥로26길 35 지하1층),정보없음,250000,37.544200,126.988000,NaN,NaN,NaN


In [15]:
df3 = pd.merge(concat_data, df1, left_on = '식당명', right_on = 'name', how='inner')

In [16]:
# 1. 보기 편하게 변수로 저장
names_df1 = set(df1['name'])      # 원본 리스트 (예: 흑백요리사 정보)
names_df2 = set(df3['식당명'])     # 크롤링한 리스트

# 2. df1(원본)에는 있는데, df2(크롤링)에는 없는 것 찾기 (누락된 식당)
not_crawled = names_df1 - names_df2

# 3. df2(크롤링)에는 있는데, df1(원본)에는 없는 것 찾기 (이름이 미묘하게 다른 경우 등)
unexpected_crawled = names_df2 - names_df1

print(f"=== [주의] df1엔 있지만 df2엔 없는 것 ({len(not_crawled)}개) ===")
# 보기 좋게 리스트로 변환해서 출력
print(list(not_crawled))

print(f"\n=== [참고] df2엔 있지만 df1엔 없는 것 ({len(unexpected_crawled)}개) ===")
print(list(unexpected_crawled))

=== [주의] df1엔 있지만 df2엔 없는 것 (0개) ===
[]

=== [참고] df2엔 있지만 df1엔 없는 것 (0개) ===
[]


In [18]:
df3

,식당명,카테고리,평점,리뷰수,지역,전화번호,저녁가격,위도,경도,URL_Alias,서비스유형,상세페이지_URL,name,chief_info,rating,review_count,location_category,status_time,price,badge_info
0,유용욱 바베큐 연구소,바베큐,4.8,268,남영,01039934729,저녁 15만원,37.543957,126.972782,yyw,DINING,https://app.catchtable.co.kr/ct/shop/yyw?type=...,유용욱 바베큐 연구소,흑백요리사2: 바베큐연구소장,4.8,(268),남영 · 바베큐,영업중 • 12:00-21:00,점심 저녁 동일가 10 - 15만원 • 저녁 15만원,"흑백요리사2: 바베큐연구소장, 흑백요리사2, 강민경 유튜브"
1,IMOK Smoke Dining,바베큐,4.8,1488,신사,01039934729,저녁 15-22만원,37.521126,127.019125,imok,DINING,https://app.catchtable.co.kr/ct/shop/imok?type...,IMOK Smoke Dining,흑백요리사2: 바베큐연구소장,4.8,"(1,488)",신사 · 바베큐,영업중 • 12:00-21:00,점심 15-18만원 • 저녁 15-22만원,"흑백요리사2: 바베큐연구소장, 동네의 고수, Taste of Seoul, 흑백요리사2"
2,라망시크레,컨템포러리,4.8,2709,회현,050712835380,저녁 27만원,37.559666,126.979542,lamantsecret,DINING,https://app.catchtable.co.kr/ct/shop/lamantsec...,라망시크레,흑백요리사2: 손종원,4.8,"(2,709)",회현 · 컨템포러리,영업중 • 12:00-22:00,점심 17만원 • 저녁 27만원,"흑백요리사2: 손종원, 미쉐린 1스타, 흑백요리사2"
3,Original Numbers 청담,코스요리,4.8,468,청담,050720889255,저녁 13만원,37.525096,127.040738,original_numbers,DINING,https://app.catchtable.co.kr/ct/shop/original_...,Original Numbers 청담,흑백요리사2: 삐딱한 천재,4.8,(466),청담 · 코스요리,영업중 • 12:00-22:00,점심 9.8만원 • 저녁 13만원,"흑백요리사2: 삐딱한 천재, 흑백요리사2"
4,앰배서더 서울 풀만 호빈,중식,4.7,96,장충동,050712835474,"점심, 저녁 동일가 11-40만원",37.560592,127.002094,theambassador_haobin,DINING,https://app.catchtable.co.kr/ct/shop/theambass...,앰배서더 서울 풀만 호빈,흑백요리사2: 후덕죽,4.7,(96),장충동 · 중식,영업중 • 11:30-22:00,"점심, 저녁 동일가 11-40만원","흑백요리사2: 후덕죽, 미쉐린 1스타, 흑백요리사2"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,떡산 안국,떡볶이/분식,정보없음,정보없음,서울 종로구 재동 (북촌로 14),0507-1309-1014,6000,37.579400,126.985100,NaN,NaN,NaN,떡산 안국,흑백요리사2: 떡볶이 명인,NaN,NaN,북촌 · 분식,NaN,점심 저녁 동일가 1만원 미만,흑백요리사2: 떡볶이 명인
114,아선재,한정식,4.3,정보없음,서울 강남구 대치동 (영동대로 333 지하1층),0507-1323-1114,120000,37.506200,127.063000,NaN,NaN,NaN,아선재,흑백요리사2: 그때 명셰프,NaN,NaN,대치동 · 한정식,영업중 • 11:30-22:00,점심 저녁 동일가 1 - 2만원,흑백요리사2: 그때 명셰프
115,우동 카덴,우동/일식,4.4,정보없음,서울 서대문구 연희동 (연희로 173 1층),02-337-6360,9000,37.569600,126.927800,NaN,NaN,NaN,우동 카덴,흑백요리사2: 정호영,5.0,(1),서교 · 일식,영업중 • 11:30-21:00,점심 1만원 미만 • 저녁 1 - 2만원,"흑백요리사2: 정호영, 흑백요리사2"
116,소울 SOUL,컨템퍼러리/한식,미슐랭1스타,정보없음,서울 용산구 해방촌 (신흥로26길 35 지하1층),정보없음,250000,37.544200,126.988000,NaN,NaN,NaN,소울 SOUL,흑백요리사2: 김희은,4.8,"(1,428)",해방촌 · 컨템포러리,오늘(수) 휴무,점심 17만원 • 저녁 27만원,"흑백요리사2: 김희은, 꽃 주문 서비스, 미쉐린 1스타, 흑백요리사2"


In [21]:
가게정보 = pd.read_csv('캐치테이블_가게정보2.csv')

In [22]:
가게정보.head()

,식당명,카테고리,평점,리뷰수,지역,전화번호,저녁가격,위도,경도,URL_Alias,서비스유형,상세페이지_URL,name,chief_info,rating,review_count,location_category,status_time,price,badge_info
0,유용욱 바베큐 연구소,바베큐,4.8,268,남영,1039934729,저녁 15만원,37.543957,126.972782,yyw,DINING,https://app.catchtable.co.kr/ct/shop/yyw?type=...,유용욱 바베큐 연구소,흑백요리사2: 바베큐연구소장,4.8,-268,남영 · 바베큐,영업중 • 12:00-21:00,점심 저녁 동일가 10 - 15만원 • 저녁 15만원,"흑백요리사2: 바베큐연구소장, 흑백요리사2, 강민경 유튜브"
1,IMOK Smoke Dining,바베큐,4.8,1488,신사,1039934729,저녁 15-22만원,37.521126,127.019125,imok,DINING,https://app.catchtable.co.kr/ct/shop/imok?type...,IMOK Smoke Dining,흑백요리사2: 바베큐연구소장,4.8,"-1,488",신사 · 바베큐,영업중 • 12:00-21:00,점심 15-18만원 • 저녁 15-22만원,"흑백요리사2: 바베큐연구소장, 동네의 고수, Taste of Seoul, 흑백요리사2"
2,라망시크레,컨템포러리,4.8,2709,회현,50712835380,저녁 27만원,37.559666,126.979542,lamantsecret,DINING,https://app.catchtable.co.kr/ct/shop/lamantsec...,라망시크레,흑백요리사2: 손종원,4.8,"-2,709",회현 · 컨템포러리,영업중 • 12:00-22:00,점심 17만원 • 저녁 27만원,"흑백요리사2: 손종원, 미쉐린 1스타, 흑백요리사2"
3,Original Numbers 청담,코스요리,4.8,468,청담,50720889255,저녁 13만원,37.525096,127.040738,original_numbers,DINING,https://app.catchtable.co.kr/ct/shop/original_...,Original Numbers 청담,흑백요리사2: 삐딱한 천재,4.8,-466,청담 · 코스요리,영업중 • 12:00-22:00,점심 9.8만원 • 저녁 13만원,"흑백요리사2: 삐딱한 천재, 흑백요리사2"
4,앰배서더 서울 풀만 호빈,중식,4.7,96,장충동,50712835474,"점심, 저녁 동일가 11-40만원",37.560592,127.002094,theambassador_haobin,DINING,https://app.catchtable.co.kr/ct/shop/theambass...,앰배서더 서울 풀만 호빈,흑백요리사2: 후덕죽,4.7,-96,장충동 · 중식,영업중 • 11:30-22:00,"점심, 저녁 동일가 11-40만원","흑백요리사2: 후덕죽, 미쉐린 1스타, 흑백요리사2"


In [1]:
import pandas as pd

pd.read_csv('캐치테이블_가게정보2.csv').head()

,식당명,카테고리,평점,리뷰수,지역,전화번호,저녁가격,위도,경도,URL_Alias,서비스유형,상세페이지_URL,name,chief_info,rating,review_count,location_category,status_time,price,badge_info
0,유용욱 바베큐 연구소,바베큐,4.8,268,남영,1039934729,저녁 15만원,37.543957,126.972782,yyw,DINING,https://app.catchtable.co.kr/ct/shop/yyw?type=...,유용욱 바베큐 연구소,흑백요리사2: 바베큐연구소장,4.8,-268,남영 · 바베큐,영업중 • 12:00-21:00,점심 저녁 동일가 10 - 15만원 • 저녁 15만원,"흑백요리사2: 바베큐연구소장, 흑백요리사2, 강민경 유튜브"
1,IMOK Smoke Dining,바베큐,4.8,1488,신사,1039934729,저녁 15-22만원,37.521126,127.019125,imok,DINING,https://app.catchtable.co.kr/ct/shop/imok?type...,IMOK Smoke Dining,흑백요리사2: 바베큐연구소장,4.8,"-1,488",신사 · 바베큐,영업중 • 12:00-21:00,점심 15-18만원 • 저녁 15-22만원,"흑백요리사2: 바베큐연구소장, 동네의 고수, Taste of Seoul, 흑백요리사2"
2,라망시크레,컨템포러리,4.8,2709,회현,50712835380,저녁 27만원,37.559666,126.979542,lamantsecret,DINING,https://app.catchtable.co.kr/ct/shop/lamantsec...,라망시크레,흑백요리사2: 손종원,4.8,"-2,709",회현 · 컨템포러리,영업중 • 12:00-22:00,점심 17만원 • 저녁 27만원,"흑백요리사2: 손종원, 미쉐린 1스타, 흑백요리사2"
3,Original Numbers 청담,코스요리,4.8,468,청담,50720889255,저녁 13만원,37.525096,127.040738,original_numbers,DINING,https://app.catchtable.co.kr/ct/shop/original_...,Original Numbers 청담,흑백요리사2: 삐딱한 천재,4.8,-466,청담 · 코스요리,영업중 • 12:00-22:00,점심 9.8만원 • 저녁 13만원,"흑백요리사2: 삐딱한 천재, 흑백요리사2"
4,앰배서더 서울 풀만 호빈,중식,4.7,96,장충동,50712835474,"점심, 저녁 동일가 11-40만원",37.560592,127.002094,theambassador_haobin,DINING,https://app.catchtable.co.kr/ct/shop/theambass...,앰배서더 서울 풀만 호빈,흑백요리사2: 후덕죽,4.7,-96,장충동 · 중식,영업중 • 11:30-22:00,"점심, 저녁 동일가 11-40만원","흑백요리사2: 후덕죽, 미쉐린 1스타, 흑백요리사2"
